In [ ]:
%matplotlib ipympl

In [ ]:
from lasy.profiles.gaussian_profile import GaussianProfile
from lasy.profiles.longitudinal import LongitudinalProfileFromData
from lasy.profiles.transverse import TransverseProfileFromData
from lasy.profiles.combined_profile import CombinedLongitudinalTransverseProfile
from lasy.laser import Laser

import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import numpy as np
import pandas as pd

In [ ]:
wavelength     = 815e-9  # Laser wavelength in meters
polarization   = (1, 0)  # Linearly polarized in the x direction
energy_J       = 24      # Pulse energy in Joules

# Measured Profile (20-24J)

## Longitudinal (Time)

In [ ]:
df_intensity = pd.read_csv("FROG_analysis_2023-06-15/23_0615Scan007Frog-HPD-1Et.txt", delimiter="\t")
#display(df_intensity)

compressor_setting_mm = 10.8
column_Re = f"Re({compressor_setting_mm}mm)"
column_Im = f"Im({compressor_setting_mm}mm)"

# time
fs = 1e-15
time_s = df_intensity["[fs]"].values * fs

# E_complex(t)
Et_Re = df_intensity[column_Re].values
Et_Im = df_intensity[column_Im].values
Et_complex = Et_Re + 1j * Et_Im

# retrieve Intensity and Phase
Et_intensity = np.abs(Et_complex)**2
Et_phase = np.arctan(Et_Im, Et_Re)

# shift to peak intensity at t=0
peak_index = np.argmax(Et_intensity)
time_s -= time_s[peak_index]

print(time_s, Et_phase.shape, Et_phase.dtype)

In [ ]:
fs = 1e-15
longitudinal_data = {
    "datatype": "temporal",
    "axis": time_s,
    "intensity": Et_intensity,
    "phase": Et_phase,
    "wavelength": wavelength
}

longitudinal_profile = LongitudinalProfileFromData(
    longitudinal_data,
    lo=-200 * fs,
    hi=200 * fs
)

In [ ]:
longitudinal_profile.evaluate(time_s)

## Transverse Mode

In [ ]:
!conda install -c conda-forge -y scikit-image

In [ ]:
import skimage

# Define the transverse profile of the laser pulse
img_url = "https://user-images.githubusercontent.com/27694869/228038930-d6ab03b1-a726-4b41-a378-5f4a83dc3778.png"
intensityData = skimage.io.imread(img_url)

# data cleaning: remove negative values
intensityData[intensityData < 2.1] = 0
# data cleaning: normalize values
pixel_calib = 0.186e-6
lo = (
    -intensityData.shape[0] / 2 * pixel_calib,
    -intensityData.shape[1] / 2 * pixel_calib,
)
hi = (
    intensityData.shape[0] / 2 * pixel_calib,
    intensityData.shape[1] / 2 * pixel_calib,
)
# data cleaning: zoom/clip (if needed)

In [ ]:
cal = 0.143 # spatial calibration mode imager
# zoom/clip
x_i_clip = [100, -100]
y_i_clip = [100, -100]

transverse_data = np.loadtxt("Mode-im/mode-im_scan10_shot25_fluencemap.csv", delimiter=",")
print(transverse_data.shape)

# data cleaning: remove negative values
transverse_data[transverse_data<1e4] = 0
# data cleaning: normalize
#transverse_scale = np.max(transverse_data)
#transverse_data /= transverse_scale
# data cleaning: zoom/clip
x_i_clip = [150, -150]
y_i_clip = [150, -150]
transverse_data = transverse_data[
    x_i_clip[0]:x_i_clip[1],
    y_i_clip[0]:y_i_clip[1]
]

# axes
transverse_x_mu = cal * np.arange(0, transverse_data.shape[1], 70)
transverse_y_mu = cal * np.arange(0, transverse_data.shape[0], 70)
transverse_y_mu

In [ ]:
fig,ax = plt.subplots()
cax = ax.imshow(
    transverse_data,
    aspect="auto",
    extent=[
        transverse_x_mu[0], transverse_x_mu[-1],
        transverse_y_mu[0], transverse_y_mu[-1]
    ],
    #norm=LogNorm(),
)
# Add a colorbar
color_bar = fig.colorbar(cax)
# Add a label to the colorbar
color_bar.set_label(r"Fluence / J/cm$^2$")
ax.set_xlabel(r"x / $\mu$m")
ax.set_ylabel("y / um")

# Set the tick positions
#ax.set_xticks(transverse_x_mu)
#ax.set_yticks(transverse_y_mu)

# Set the tick labels
#ax.set_xticklabels([f"{i:.{0}f}" for i in transverse_x_mu])
#ax.set_yticklabels([f"{j:.{0}f}" for j in transverse_y_mu])

plt.show()

In [ ]:
mu = 1e-6
transverse_profile = TransverseProfileFromData(
    transverse_data,
    [transverse_x_mu[0]  * mu, transverse_y_mu[0]  * mu],
    [transverse_x_mu[-1] * mu, transverse_y_mu[-1] * mu]
)

# Transversal Profile Denoising
See https://github.com/LASY-org/lasy/blob/13f0e4515493deca36c1375be1d9e83c7e379d42/examples/example_modal_decomposition_data.py

In [ ]:
from lasy.utils.mode_decomposition import hermite_gauss_decomposition
from lasy.profiles.transverse.hermite_gaussian_profile import (
    HermiteGaussianTransverseProfile,
)

# Calculate the decomposition into hermite-gauss modes
n_x_max = 20
n_y_max = 20
modeCoeffs, waist = hermite_gauss_decomposition(
    transverse_profile, n_x_max=n_x_max, n_y_max=n_y_max, res=0.5e-6
)

## Combine

In [ ]:
# original transverse data
org_laser_profile = CombinedLongitudinalTransverseProfile(
    wavelength=wavelength,
    pol=polarization,
    laser_energy=energy_J,
    long_profile=longitudinal_profile,
    trans_profile=transverse_profile
)

In [ ]:
org_laser_profile.laser_energy

In [ ]:
# denoised transverse data

# Reconstruct the pulse using a series of hermite-gauss modes
for i, mode_key in enumerate(list(modeCoeffs)):
    tmp_transverse_profile = HermiteGaussianTransverseProfile(
        waist, mode_key[0], mode_key[1]
    )
    if i == 0:
        laser_profile = modeCoeffs[
            mode_key
        ] * CombinedLongitudinalTransverseProfile(
            wavelength, polarization, energy_J, longitudinal_profile, tmp_transverse_profile
        )
    else:
        laser_profile += modeCoeffs[
            mode_key
        ] * CombinedLongitudinalTransverseProfile(
            wavelength, polarization, energy_J, longitudinal_profile, tmp_transverse_profile
        )

In [ ]:
laser_profile.laser_energy

## Plot Denoised Profile

In [ ]:
from mpl_toolkits.axes_grid1 import make_axes_locatable

# Plotting the results
x = np.linspace(-5 * waist, 5 * waist, 500)
X, Y = np.meshgrid(x, x)

fig, ax = plt.subplots(1, 3, figsize=(12, 4), tight_layout=True)

pltextent = (np.min(x) * 1e6, np.max(x) * 1e6, np.min(x) * 1e6, np.max(x) * 1e6)
prof1 = np.abs(org_laser_profile.evaluate(X, Y, 0)) ** 2
divider0 = make_axes_locatable(ax[0])
ax0_cb = divider0.append_axes("right", size="5%", pad=0.05)
pl0 = ax[0].imshow(prof1, cmap="magma", extent=pltextent, vmin=0, vmax=np.max(prof1))
cbar0 = fig.colorbar(pl0, cax=ax0_cb)
cbar0.set_label("Intensity (norm.)")
ax[0].set_xlabel("x (micron)")
ax[0].set_ylabel("y (micron)")
ax[0].set_title("Original Profile")

prof2 = np.abs(laser_profile.evaluate(X, Y, 0)) ** 2
divider1 = make_axes_locatable(ax[1])
ax1_cb = divider1.append_axes("right", size="5%", pad=0.05)
pl1 = ax[1].imshow(prof2, cmap="magma", extent=pltextent, vmin=0, vmax=np.max(prof1))
cbar1 = fig.colorbar(pl1, cax=ax1_cb)
cbar1.set_label("Intensity (norm.)")
ax[1].set_xlabel("x (micron)")
ax[1].set_ylabel("y (micron)")
ax[1].set_title("Reconstructed Profile")


prof3 = (prof1 - prof2) / np.max(prof1)
divider2 = make_axes_locatable(ax[2])
ax2_cb = divider2.append_axes("right", size="5%", pad=0.05)
pl2 = ax[2].imshow(100 * np.abs(prof3), cmap="magma", extent=pltextent, vmin=0, vmax=5)
cbar2 = fig.colorbar(pl2, cax=ax2_cb)
cbar2.set_label("|Intensity Error| (%)")
ax[2].set_xlabel("x (micron)")
ax[2].set_ylabel("y (micron)")
ax[2].set_title("Error")

fig.suptitle(
    "Hermite-Gauss Reconstruction using n_x_max = %i, n_y_max = %i" % (n_x_max, n_y_max)
)
plt.show()

### RT

In [ ]:
dimensions     = "rt"                               # Use 3D geometry
lo             = (0, -7*pulse_duration)           # Lower bounds of the simulation box
hi             = (5*spot_size, 10*pulse_duration)  # Upper bounds of the simulation box
num_points     = (300, 500)                     # Number of points in each dimension

laser_rt_org = Laser(dimensions, lo, hi, num_points, org_laser_profile)

In [ ]:
plt.figure()
laser_rt_org.show()

In [ ]:
dimensions     = "rt"                               # Use 3D geometry
lo             = (0, -7*pulse_duration)           # Lower bounds of the simulation box
hi             = (5*spot_size, 10*pulse_duration)  # Upper bounds of the simulation box
num_points     = (300, 500)                     # Number of points in each dimension

# hack: add laser_energy attribute
laser_profile.laser_energy = energy_J

laser_rt = Laser(dimensions, lo, hi, num_points, laser_profile)

In [ ]:
plt.figure()
laser_rt.show()

### XYT

In [ ]:
dimensions     = "xyt"                               # Use 3D geometry
lo             = (-12.0e-6, -12.0e-6, -7*pulse_duration)           # Lower bounds of the simulation box
hi             = ( 12.0e-6,  12.0e-6, 10*pulse_duration)  # Upper bounds of the simulation box
#lo             = (transverse_x_mu[0] *mu, transverse_y_mu[0] *mu, -7*pulse_duration)           # Lower bounds of the simulation box
#hi             = (transverse_x_mu[-1]*mu, transverse_y_mu[-1]*mu, 10*pulse_duration)  # Upper bounds of the simulation box
num_points     = (300, 300, 500)                     # Number of points in each dimension
#num_points     = (50, 50, 200)                     # low res for quick tests

laser_xyt = Laser(dimensions, lo, hi, num_points, laser_profile)

In [ ]:
plt.figure()
laser_xyt.show()

# Propagate Backwards from Focus for Initialization in Simulation

In [ ]:
profile_focal_distance = 24.0e-6
laser_xyt.propagate(-profile_focal_distance)  # Propagate the pulse upstream of the focal plane

In [ ]:
plt.figure()
laser_xyt.show()

In [ ]:
file_prefix    = "BELLAiP2_24J_24um_to_focus"

laser_xyt.write_to_file(file_prefix, file_format="h5")